In [13]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import os

In [14]:
# 设定超参数
batch_size = 128
learning_rate = 0.001
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
# (5000, 1, 32, 32)的numpy数组
data = np.load("data4D.npy")

In [8]:
data_tensor = torch.from_numpy(data)
dataset = TensorDataset(data_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [10]:
# 定义自编码器结构
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


# 初始化自编码器并将其移至相应的设备（CPU或GPU）
model = Autoencoder().to(device)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练自编码器
for epoch in range(num_epochs):
    for data in dataloader:
        img = data[0].to(device)
        # 前向传播
        output = model(img)
        loss = criterion(output, img)

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 输出日志信息
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training completed.")

[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.


Epoch [1/100], Loss: 0.0488
Epoch [2/100], Loss: 0.0557
Epoch [3/100], Loss: 0.0473
Epoch [4/100], Loss: 0.0513
Epoch [5/100], Loss: 0.0448
Epoch [6/100], Loss: 0.0660
Epoch [7/100], Loss: 0.0583
Epoch [8/100], Loss: 0.0504
Epoch [9/100], Loss: 0.0578
Epoch [10/100], Loss: 0.0515
Epoch [11/100], Loss: 0.0588
Epoch [12/100], Loss: 0.0513
Epoch [13/100], Loss: 0.0543
Epoch [14/100], Loss: 0.0490
Epoch [15/100], Loss: 0.0385
Epoch [16/100], Loss: 0.0418
Epoch [17/100], Loss: 0.0427
Epoch [18/100], Loss: 0.0563
Epoch [19/100], Loss: 0.0530
Epoch [20/100], Loss: 0.0448
Epoch [21/100], Loss: 0.0664
Epoch [22/100], Loss: 0.0426
Epoch [23/100], Loss: 0.0303
Epoch [24/100], Loss: 0.0441
Epoch [25/100], Loss: 0.0621
Epoch [26/100], Loss: 0.0435
Epoch [27/100], Loss: 0.0424
Epoch [28/100], Loss: 0.0410
Epoch [29/100], Loss: 0.0439
Epoch [30/100], Loss: 0.0374
Epoch [31/100], Loss: 0.0423
Epoch [32/100], Loss: 0.0345
Epoch [33/100], Loss: 0.0466
Epoch [34/100], Loss: 0.0347
Epoch [35/100], Loss: 0

In [11]:
# 保存模型权重
model_path = "autoencoder_weights.pth"
torch.save(model.state_dict(), model_path)
print("Model weights saved.")

Model weights saved.


In [12]:
# 加载模型权重
loaded_model = Autoencoder().to(device)
loaded_model.load_state_dict(torch.load(model_path))
loaded_model.eval()
print("Model weights loaded.")


Model weights loaded.
